In [1]:
#Importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.cbook import boxplot_stats  
import statsmodels.api as sm
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, cross_val_predict
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import ensemble 
from sklearn.metrics import classification_report, accuracy_score

In [2]:
data = pd.read_csv("emails_V2.csv")

In [3]:
data.dropna(subset = ["spam"], inplace=True)

In [4]:
data['text'] = data['text'].str.replace('Subject: ', '')

In [5]:
data.head()

,text,spam
0,naturally irresistible your corporate identity...,1.0
1,the stock trading gunslinger fanny is merrill...,1.0
2,unbelievable new homes made easy im wanting t...,1.0
3,4 color printing special request additional i...,1.0
4,"do not have money , get software cds from here...",1.0


In [6]:
data.spam.value_counts()

1.0    1368
0.0     675
Name: spam, dtype: int64

In [7]:
data.spam.value_counts() / len(data)

1.0    0.669604
0.0    0.330396
Name: spam, dtype: float64

In [8]:
length_text = data.text.str.len()

In [9]:
length_text

0       1475
1        589
2        439
3        491
4        226
        ... 
2038     376
2039     142
2040     441
2041    2008
2042     464
Name: text, Length: 2043, dtype: int64

In [12]:
txt = data['text'][0:3]

In [13]:
txt

0    naturally irresistible your corporate identity...
1    the stock trading gunslinger  fanny is merrill...
2    unbelievable new homes made easy  im wanting t...
Name: text, dtype: object

In [14]:
#instantiate the class
cv = CountVectorizer()

In [15]:
#tokenize and build vocab
cv.fit(txt)

CountVectorizer()

In [16]:
print(cv.vocabulary_)

{'naturally': 142, 'irresistible': 111, 'your': 226, 'corporate': 48, 'identity': 101, 'lt': 128, 'is': 112, 'really': 175, 'hard': 88, 'to': 206, 'recollect': 176, 'company': 43, 'the': 200, 'market': 132, 'full': 80, 'of': 147, 'suqgestions': 193, 'and': 10, 'information': 108, 'isoverwhelminq': 114, 'but': 29, 'good': 82, 'catchy': 30, 'logo': 124, 'stylish': 191, 'statlonery': 188, 'outstanding': 156, 'website': 218, 'will': 219, 'make': 130, 'task': 198, 'much': 139, 'easier': 61, 'we': 217, 'do': 55, 'not': 145, 'promise': 169, 'that': 199, 'havinq': 90, 'ordered': 152, 'iogo': 110, 'automaticaily': 18, 'become': 20, 'world': 223, 'ieader': 102, 'it': 115, 'isguite': 113, 'ciear': 37, 'without': 222, 'products': 168, 'effective': 64, 'business': 28, 'organization': 153, 'practicable': 166, 'aim': 6, 'be': 19, 'hotat': 99, 'nowadays': 146, 'marketing': 133, 'efforts': 65, 'more': 137, 'here': 95, 'list': 122, 'clear': 38, 'benefits': 24, 'creativeness': 49, 'hand': 87, 'made': 129

In [18]:
#transform the text
vector = cv.transform(txt)
print(vector.toarray())

[[1 0 0 0 0 1 1 0 1 1 5 0 0 1 0 1 0 0 1 2 2 0 0 0 1 0 1 1 2 2 1 0 1 1 0 0
  0 1 1 0 0 1 0 3 0 2 0 1 1 1 0 1 0 0 1 2 1 0 1 0 0 1 1 0 2 1 0 0 1 0 1 0
  0 1 0 1 0 1 0 0 1 1 2 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 2 0 0
  1 1 1 1 3 1 1 2 1 0 1 0 0 0 1 0 3 1 1 1 1 1 2 1 2 2 0 0 0 1 0 2 0 0 1 0
  1 2 1 4 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 2 1 1 1 0 0 1 1 1 1 0
  1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 0 1 3 5 1 1 1 0 0 4 0 0 0 0 1 1 0 0 0
  0 3 2 6 1 1 1 1 0 3 7]
 [0 0 0 0 0 0 0 1 0 0 3 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 3 0 1 0 0 1 1
  1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1
  0 0 0 0 6 0 0 0 0 1 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0
  3 6 0 0 0 0 1 3 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 3 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 3 0 0]
 [0 1 1 1 1 0 0 0 1 0 2 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 

## Bag of words

In [24]:
txt_vec = CountVectorizer().fit_transform(data['text'])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(txt_vec, data['spam'], test_size = 0.2, 
                                                    random_state = 2, shuffle = True)

## Classification

In [26]:
classifier = ensemble.GradientBoostingClassifier(
    n_estimators = 100, #how many decision trees to build
    learning_rate = 0.5, #learning rate
    max_depth = 6
)

## Prediction

In [27]:
classifier.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.5, max_depth=6)

In [28]:
predictions = classifier.predict(X_test)

In [29]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.99      0.91      0.95       139
         1.0       0.96      1.00      0.98       270

    accuracy                           0.97       409
   macro avg       0.97      0.95      0.96       409
weighted avg       0.97      0.97      0.97       409

